In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import time

In [22]:
df = pd.read_csv('new_words.csv')
df = df[df.sim>0.6]
s_word = nltk.corpus.stopwords.words('english')
seed = input('The keyword requires expanding is: ')

The keyword requires expanding is: biofuels


In [24]:
df1 = df[df.w1 == seed].sort_values(by='sim', ascending=False).reset_index(drop=True)
cand = list(df1.w2[:500])

t1 = time.time()
## Basic cleaning

word_list = [re.findall(r"[a-zA-Z].*[a-zA-Z0-9]", i)[0] for i in cand
             if re.findall(r"[a-zA-Z].*[a-zA-Z0-9]", i)]

word_list = list(set(word_list))

word_list = [i for i in word_list if len(re.findall(r"[a-zA-Z]+", i))>1 and len(re.findall(r"[a-zA-Z]+", i)[-1])>1
             and not (re.findall(r"[a-zA-Z0-9]+", i)[0] in s_word or re.findall(r"[a-zA-Z0-9]+", i)[-1] 
                      in s_word or re.findall(r"[^a-zA-Z0-9 ]+", i))]

## Integrate keywords containing same phrases

fin3 = []
for i in range(len(word_list)-1):
    fin3.append([word_list[i]])
    for j in word_list[i+1:]:
        if word_list[i] in j:
            fin3[i].append(j)

idx = []
for i in range(len(fin3)-1):
    idx.append([i])
    for j in range(i+1, len(fin3)):
        if set(fin3[i]) - set(fin3[j]) != set(fin3[i]):
            idx[i].append(j)
idx.append([len(fin3)-1])

new_idx = idx[:]
old = 0
new = 1
while new != old:
    old = list(map(set, new_idx))
    new = list(map(set, new_idx))
    for i in range(len(new_idx)):
        if len(new_idx[i])>1:
            for j in new_idx[i][1:]:
                new[i].update(new[j])
                new[j] = set()     
    new_idx = list(map(list, new))

fin = list(map(set, fin3))
for i in range(len(new_idx)):
    if len(new_idx[i])>1:
        for j in new_idx[i][1:]:
            fin[i].update(fin[j])
            fin[j] = set()

fin = list(map(list, fin))
fin = [i for i in fin if i]

t2 = time.time()
print(t2-t1)

0.06772065162658691


In [26]:
pd.DataFrame(fin).to_csv(seed+'.csv')